# Tratamento de dados de devedores
1. O ficheiro dos devedores tem que se chamar 'devedores.xls' e o CRM 'crm.xls'
1. Definir a data da listagem
1. Carregar os ficheiros que vão ser processados selecionar com CTRL
1. Os ficheiros são gerados para esta pasta https://www.dropbox.com/sh/dcb79ge6jphck10/AABjtNch8UAbLNlmoqmpo7qOa?dl=0 já tem lá dados gerados com os ficheiros de exemplo

# Carregar os ficheiros (CRM e devedores)

In [2]:
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link, DatePicker
import xlrd
import pandas as pd
import datetime

In [3]:
import numpy as np

In [4]:
dfDevedores = pd.DataFrame()
dfCRM = pd.DataFrame()
dataListagem = datetime.datetime

In [5]:
out = Output()

In [6]:
def plot_stats():
    stat_layout = Layout(margin="10px", padding="10px", border="1px solid black",flex_flow='column', align_items='center')
    stats = [('1','2'),('2','3')]
    stats_formatted = [
        VBox([
            HTML(value=f"<strong>{title}</strong>"),
            Label(value=value)
        ], layout=stat_layout)
        for title, value in stats
    ]
    
    return HBox(stats_formatted, layout=Layout(flex_flow='row', align_items='center'))

In [7]:
def plot_gpx():
    stats = plot_stats()
    
    display(stats)

In [8]:
from dateutil import rrule
import datetime
def weeks_between(start_date, end_date):
    weeks = rrule.rrule(rrule.WEEKLY, dtstart=start_date, until=end_date)
    return weeks.count()

def tratamento():
    global dfDevedores
    global dfCRM
    #limpar todos excepto que têm pelo menos 1 valor non-NA = 
    dfDevedores.dropna(axis=0,thresh=1,inplace=True)
    #limpar todas as colunas que so têm NaNs how='all'
    dfDevedores.dropna(axis=1,how='all',inplace=True)

    #limpar as colunas sem dados
    dfDevedores.drop(axis=1,inplace=True,labels=['Unnamed: 3','Unnamed: 4','Unnamed: 5'])
    colsDevedores=['tratar', 'numCli', 'nome', 'telemovel','venc', 'ultVisita', 'idade', 'valor']
    dfDevedores.columns=['tratar', 'idCli', 'nome', 'telemovel','venc', 'ultVisita', 'idade', 'valor']

    dfDevedores=dfDevedores.reset_index()
    dfDevedores.drop(axis=1,inplace=True,labels=['index'])
    dfDevedores.to_excel('output/devedores_novo.xls')
    #Guarda o último valor antes do número do cliente
    # Vamos converter para texto
    dfDevedores.tratar = dfDevedores.tratar.astype('str') 

    pacote = 'nan'
    for index, item in dfDevedores.iterrows():
        if item['tratar'] != 'nan':
            #print(item['tratar'])
            pacote = item['tratar']
        if (item['idCli']!=np.nan) & (pacote != 'nan'):
            #print('cliente',item['idCli'])
            dfDevedores.at[index,'tratar']=pacote

    #limpar todos que têm nan no idCliente
    dfDevedores.dropna(subset=['idCli'],axis=0,inplace=True)
    dfDevedores = dfDevedores[['idCli', 'nome', 'telemovel', 'venc', 'ultVisita', 'idade','valor','tratar']]
    dfDevedores.columns=['idCli', 'nome', 'telemovel','venc', 'ultVisita', 'idade', 'valor','servico']

    # Retirar as casas decimais
    dfDevedores.idCli = dfDevedores.idCli.astype(int)
    # Juntar os dataframes
    dfDevedores=pd.merge(dfDevedores,dfCRM[['Nome','Status']], left_on="nome",right_on="Nome")
    #limpar todos que têm nan no idCliente
    dfDevedores.dropna(subset=['Nome'],axis=0,inplace=True)
    #Calcular as semanas
    dfDevedores['semanas'] = 0
    for index, item in dfDevedores.iterrows():
        dfDevedores.at[index,'semanas']=weeks_between(item['venc'],dataListagem.value)
    #Calcular as quinzenas
    dfDevedores['diaSemana']=dfDevedores['venc'].apply(lambda x: x.weekday())
    dfDevedores['quinzena']=dfDevedores['semanas'].apply(lambda x: round(x/2,0))
    #Definir itens para não multiplicar
    naoMultiplica = ['Pacote LT c/12M 2 meses','Pacote LT c/12MNUTRIÇĂO 2 meses', 'Pacote LT c/24M 2 meses',
                 'Pacote LT c/24MNUTRIÇĂO 2 meses','SeguroELITE']
    #Calcular o valor em dívida
    valores=[]
    nomes = dfDevedores['nome'].unique()
    for nome in nomes:
        dfDevedores['total'] = 0
        valor = 0
        for index, item in dfDevedores.loc[dfDevedores['nome']==nome].iterrows():
            if item['nome'] == nome:
                if item['servico'] in naoMultiplica:
                    #print('não multiplica')
                    valor = valor + item['valor']*1
                else: #Multiplica pelas quinzenas em atraso
                    #print('multiplica',item['valor'],item['quinzena'])
                    valor = valor + item['valor']*item['quinzena']
        valores.append(valor)          
    #Criar um data frame com os valores em dívida
    dfValores = pd.DataFrame(data={'Nome2': nomes, 'valor': valores})
    #Juntar tudo
    dfDevedores=pd.merge(dfDevedores,dfValores[['Nome2','valor']], left_on="nome",right_on="Nome2",how='outer')
    #Apagar as colunas que não interessam
    dfDevedores.drop(columns=[ 'total', 'Nome2','idade','Nome'],axis=1,inplace=True)
    #Mudar os nomes das colunas
    dfDevedores.columns = ['idCli', 'nome', 'telemovel', 'venc', 'ultVisita', 'valor', 'servico',
                           'Status', 'semanas', 'diaSemana', 'quinzena', 'total']
    #Gravar um excel    
    dfDevedores.to_excel('output/devedores_novo.xls')

In [11]:
def show_uploader():
    uploader = FileUpload(accept='.xls', multiple=True)

    def handle_upload(change):
        out.clear_output()
        global dfDevedores
        global dfCRM
        
        # Isto é um dict dentro de um dict para cada ficheiro
        # Vamos selecionar o primeiro dict e depois o segundo
        with open("output/devedores_novo.xls", "w+b") as i:
            i.write(uploader.value.get('devedores.xls').get('content'))

        workbook = xlrd.open_workbook('output/devedores_novo.xls',encoding_override='cp1250')
        dfDevedores=pd.read_excel(workbook)
        dfDevedores.to_excel('output/devedores_novo.xls')
        with out:
            print('carregado devedores', dfDevedores.shape)
            #plot_gpx()
        
            
        dfCRM=pd.read_csv("crm.xls", delimiter='\t', encoding='latin_1', engine='python',error_bad_lines=False)
        dfCRM.to_excel('output/crm_novo.xls',encoding='utf-8')
        with out:
            print('carregado crm', dfCRM.shape)
            #plot_gpx()

        with out:
            print('tratamento...')
            #plot_gpx()
        tratamento()
        with out:
            print('terminei!')
            #plot_gpx()
    uploader.observe(handle_upload, names='value')

    global dataListagem
    dataListagem=DatePicker(description='Data',disabled=False)
    display (dataListagem)
    display(uploader)

show_uploader()    

DatePicker(value=None, description='Data')

FileUpload(value={}, accept='.xls', description='Upload', multiple=True)

In [10]:
out

Output()